In [2]:
import sys
sys.path.append('../')

import cqt
import cqt.env as env
import cqt.ledger.ledger as ledger
import cqt.strats.strategy as stg
import cqt.strats.strategy_double_dip as sdd
import pandas as pd
import pickle

from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

In [3]:
file_btc_eth = open('../data/pickle/model_btc_eth.pickle','rb')
env_btc_eth = pickle.load(file_btc_eth)

ModuleNotFoundError: No module named 'cqt.model'

In [ ]:
# init portfolio

cash = 100000
assets = {'btc': 0, 'eth': 0}
ini_pfo = pfo.Portfolio(assets, cash)

In [ ]:
# Long/Short Average

ls_rule = {'method' : 'moving_average', 
           'window_size' : [5, 15], 
           'tolerance_up' : 0.0, 
           'tolerance_down' : 0.0,
           'damping_factor' : 1.0}
strats = stg_ls.StrategyLongShortAverage(model_btc_eth, ini_pfo, ls_rule)
start_date_str = '2018-02-01'
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
report = strats.back_testing(start_date)

plt.plot(report['date'], report['total'])
plt.plot(report['date'], report['btc_price'] * cash / report['btc_price'][0])
plt.show()
plt.plot(report['date'], report['pnl'])
plt.show()

In [ ]:
comp_btc = model_btc_eth.get_component('btc')
comp_btc.plot_close_moving_average([5,30])

In [ ]:
class StrategyTest(stg.Strategy):
    def apply_event_logic(self, time, prtf):
        btc = 'btc'
        if self.asset_model.has_component(btc):            
            comp_btc = self.asset_model.get_component(btc)
            ind_btc = comp_btc.signal_double_dip(time, self.rules)
            price_btc = comp_btc.get_price_close(time)
            
            if ind_btc == 1:
                prtf.buy(btc, price_btc)
            else:
                prtf.sell(btc, price_btc)
            
            asset_prices = self.asset_model.get_prices_close(time)
            
            if prtf.value(asset_prices) > 1.05 * 100000:
                prtf.set_aside()
            
            
        return prtf

In [ ]:
dd_rule = {'method' : 'moving_average', 'window_size' : [7, 20], 'tolerance_up' : 0.0, 'tolerance_down' : 0.0}
strats = StrategyTest(model_btc_eth, ini_pfo, dd_rule)
start_date_str = '2018-01-01'
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
report = strats.back_testing(start_date)

plt.plot(report['date'], report['total'])
plt.plot(report['date'], report['btc_price'] * cash / report['btc_price'][0])
plt.show()
plt.plot(report['date'], report['pnl'])
plt.show()

In [ ]:
print(report)